# Homework1: Finish 2D convolution/filtering by your self. 
What you are supposed to do can be described as "median blur", which means by using a sliding window on an image, your task is not going to do a normal convolution, but to find the median value within that crop.

You can assume your input has only one channel. (a.k.a a normal 2D list/vector)
And you do need to consider the padding method and size. There are 2 padding ways: REPLICA & ZERO. When "REPLICA" is given to you, the padded pixels are same with the border pixels. E.g is [1 2 3] is your image, the padded version will be [(...1 1) 1 2 3 (3 3...)] where how many 1 & 3 in the parenthesis depends on your padding size. When "ZERO", the padded version will be [(...0 0) 1 2 3 (0 0...)]

Assume your input's size of the image is W x H, kernel size's m x n. You may first complete a version with O(W·H·m·n log(m·n)) to O(W·H·m·n·m·n)). Follow up 1: Can it be completed in a shorter time complexity?

## 1.中值滤波的实现
中值滤波是常用的一种滤波方式，属于非线性的滤波。它的原理就是将窗口像素排序，取中值，然后移动窗口，不断重复取中值的过程。

In [9]:
import cv2
import numpy as np
import math

In [10]:
# 用于彩色图像
def medianBlur(img, kernel=(3,3), padding_way='ZERO'):
    '''
    img: 图像名称
    kernel is List of List: [m, n] 所要输入的kernel含义不是特别明确，先理解为中值滤波kernel的尺寸
    padding_way a string: REPLICA(填充元像素与边缘像素相同); ZERO(填充像素为0)
    '''
    #对输入的padding方式进行检查
    padding_way = padding_way.upper()
    allowed_padding = ['ZERO', 'REPLICA']
    if padding_way not in allowed_padding:
        raise ValueError('The padding argument must be one of "ZERO", "REPLICA". Received: ' + str(padding))
    #设置与img大小相同的全0矩阵，用于接收中值操作后的值，形成中值滤波操作后的结果
    img_after_Blur = np.zeros_like(img)
    #保存图像的形状
    img_shape = img.shape
    #图像填充，padding，使用np中的np.lib.pad函数
    padding = ((math.ceil(kernel[0]/2),),(math.ceil(kernel[1]/2),))
    #若img是三通道的彩色图像，则padding矩阵增加一个维度，但是不做paddding操作
    if len(img_shape) == 3:
        padding = padding + ((0,),)
    #不同的padding方式
    if padding_way == 'REPLICA':
        img = np.lib.pad(img, padding, mode='edge')
    if padding_way == 'ZERO':
        img = np.lib.pad(img, padding, mode='constant', constant_values = 0)
    #取kernel对应于padding后img的中值，形成新图像，即中值滤波
    for j in range(img_shape[1]):
        for i in range(img_shape[0]):
            kernel_window = img[i+1:i+padding[0][0]*2+2, j+1:j+padding[1][-1]*2+2]
            img_after_Blur[i, j , :] = np.median(kernel_window, axis=(0,1))
    return img_after_Blur

### 上述代码参考了优秀作业1. 但是遍历方法求中值，速度还是有点慢，下面的实例也可以发现该代码实现中值滤波速度很慢，512x512的图像需要25秒。可以考虑答疑课中提到的针对求中值的加速办法进行改进
### 由于图像通过opencv读入后，类型为numpy.ndarray，用np.median函数求中位数是较好的选择

In [11]:
#测试1
path = './lenna.jpg'
img = cv2.imread(path)
print(img.shape)

(512, 512, 3)


In [12]:
a = medianBlur(img)
new_path = './lenna_blurred.jpg'
cv2.imwrite(new_path, a)
cv2.imshow('a', a)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

In [1]:
#opencv中还可以使用cv2.copyMakeBorder进行pandding操作
'''
    if padding_way == 'REPLICA':
        padding_my_img = cv2.copyMakeBorder(my_img, 1, 1, 1, 1, cv2.BORDER_REPLICATE)
    elif padding_way == 'ZERO':
        padding_my_img = cv2.copyMakeBorder(my_img, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)
    my_img_medianBlur = cv2.medianBlur(padding_my_img, kernel_size)
'''
#opencv中，可以直接调用cv2.medianBlur进行中值滤波

"\n    if padding_way == 'REPLICA':\n        padding_my_img = cv2.copyMakeBorder(my_img, 1, 1, 1, 1, cv2.BORDER_REPLICATE)\n    elif padding_way == 'ZERO':\n        padding_my_img = cv2.copyMakeBorder(my_img, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)\n    my_img_medianBlur = cv2.medianBlur(padding_my_img, kernel_size)\n"

## 2.中值滤波加速算法的实现
利用窗口每次平移时，没有移出窗口的像素点还是排好序的，因此，只需要把新加入的像素点插入到其中即可完成排序。此外，由于我们并不需要一个完整的排序数列， 只需要找到中值就可以了。

基于上面两点，快速中值滤波采用的是直方图的方式来统计像素点，即横坐标是像素点的值，纵坐标是窗口中像素点的个数。设置一个“光标”，在横轴上左右移动，当两边像素点相等时，对应的像素点就是要找的中值。然后移动窗口，更新直方图，再次移动“光标”，找到中值，不断反复。

需要注意的是，“光标”的每次移动，都只能移动在数目不为 0 的像素点上，否则，取到的中值很可能不在这个窗口中。

加速中值滤波的具体步骤如下：
1. 第一步：
设置门限th = N*M/2。门限是用来判断像素点是否是中值的，它是窗口大小的一半，如果不知道有啥用，看第五步的用法。

2. 第二步：
将窗口移动到一个新行的开始，建立窗口像素的直方图，通过直方图确定中值 med，记下亮度小于或等于 med 的像素数目到变量 n 中。

3. 第三步：
对于最左列的每个像素，去掉每一个元素，并将直方图中的相应的数值更新，然后更新n的值。

4. 第四步：
同理，与第三步一样，对于最右列的每个像素，增加每一个元素的值，并将直方图中的相应的数值进行更新。

5. 第五步：
然后判断n的值与门限 th 的大小。如果 n > th，则将 med 进行递减操作；如果n < th 则将 med 进行递增操作。直到 n 超过 th 为止。得到的 med 就是需要的中值。

参考自[博客](https://blog.csdn.net/rocketeerLi/article/details/88017306)

In [13]:
import cv2
import numpy as np
import math

In [18]:
# 彩色图像
def medianBlur_turbo(img, kernel=(3,3), padding_way='ZERO'):
    '''
    img: 图像名称
    kernel is List of List: [m, n] 所要输入的kernel含义不是特别明确，先理解为中值滤波kernel的尺寸
    padding_way a string: REPLICA(填充元像素与边缘像素相同); ZERO(填充像素为0)
    '''
    # 对输入的padding方式进行检查
    padding_way = padding_way.upper()
    allowed_padding = ['ZERO', 'REPLICA']
    if padding_way not in allowed_padding:
        raise ValueError('The padding argument must be one of "ZERO", "REPLICA". Received: ' + str(padding))
    # 设置与img大小相同的全0矩阵，用于接收中值操作后的值，形成中值滤波操作后的结果
    img_after_Blur = np.zeros_like(img)
    # 保存图像的形状
    img_shape = img.shape
    # 图像填充，padding，使用np中的np.lib.pad函数
    padding = ((math.ceil(kernel[0]/2),),(math.ceil(kernel[1]/2),))
    # 若img是三通道的彩色图像，则padding矩阵增加一个维度，但是不做paddding操作
    if len(img_shape) == 3:
        padding = padding + ((0,),)
    #不同的padding方式
    if padding_way == 'REPLICA':
        img = np.lib.pad(img, padding, mode='edge')
    if padding_way == 'ZERO':
        img = np.lib.pad(img, padding, mode='constant', constant_values = 0)
        
    # 以上步骤与原中值滤波函数相同，包括读取pandding方式、进行pandding操作
    # 由于是彩色图片，要绘制直方图需要按照BGR三个通道分别进行
    B, G, R = cv2.split(img)
    # 设置门限
    th = math.ceil(kernel[0]*kernel[1]/2)
    # 对蓝色通道进行中值滤波
    # 创建保存直方图的数组
    H = np.zeros(256, dtype=int)
    for row in range(1, len(B) - 1):
        # 到达一个新的行，对直方图进行初始化
        H = np.zeros(256, dtype=int)
        # 求中值
        med = np.uint8( np.median(B[row - 1 : row + kernel[0]-1, 0:kernel[1]]) )
        n = 0 
        # 绘制直方图
        for i in range(-1, kernel[0]-1) :
            for j in range(0, kernel[1]) :
                H[B[row+i][j]] = H[B[row+i][j]] + 1
                if B[row+i][j] <= med:
                    n = n + 1
        for col in range(1, len(B[row]) - 1) :
            # 第一列
            if col == 1 :
                None
            # 移到下一列
            else :
                # 更新直方图 并计算 n 的值
                for i in range(-1, kernel[0]-1) :
                    # 对左列元素 值减一 
                    H[B[row+i][col-2]] = H[B[row+i][col-2]] - 1
                    if B[row+i][col-2] <= med :
                        n = n - 1
                    # 对右列元素 值加一
                    H[B[row+i][col+1]] = H[B[row+i][col+1]] + 1
                    if B[row+i][col+1] <= med :
                        n = n + 1
                # 重新计算中值
                if n > th :
                    while n > th :
                        if med == 0 :
                            break
                        n = n - H[med]
                        med = med - 1
                elif n < th :
                    while n < th :
                        med = med + 1
                        n = n + H[med]
            sum = 0
            for k in range(med + 1) :
                sum = sum + H[k]
            # 更新中值后的直方图
            H[B[row][col]] = H[B[row][col]] - 1
            if med < B[row][col] :
                n = n + 1
            B[row][col] = med
            H[med] = H[med] + 1
    # 对 绿色通道 进行中值滤波
    H = np.zeros(256, dtype=int)    # 直方图
    for row in range(1, len(G) - 1) :
        # 到达一个新的行 初始化
        H = np.zeros(256, dtype=int)    # 直方图
        # 求中值
        med = np.uint8(np.median(G[row - 1 : row + 2, 0:3]))
        if med == -128 :
            print(G[row - 1 : row + 2, 0:3])
        n = 0
        for i in range(-1, 2) :
            for j in range(0, 3) :
                H[G[row+i][j]] = H[G[row+i][j]] + 1
                if G[row+i][j] <= med :
                    n = n + 1
        for col in range(1, len(G[row]) - 1) :
            if col == 1 :
                None
            # 移到下一列
            else :
                # 更新直方图 并计算 n 的值
                for i in range(-1, 2) :
                    # 对左列元素 值减一 
                    H[G[row+i][col-2]] = H[G[row+i][col-2]] - 1
                    if G[row+i][col-2] <= med :
                        n = n - 1
                    # 对右列元素 值加一
                    H[G[row+i][col+1]] = H[G[row+i][col+1]] + 1
                    if G[row+i][col+1] <= med :
                        n = n + 1
                # 重新计算中值
                if n > th :
                    while n > th :
                        if med == 0 :
                            break
                        n = n - H[med]
                        med = med - 1
                elif n < th :
                    while n < th :
                        med = med + 1
                        n = n + H[med]
            # 更新中值后的直方图
            H[G[row][col]] = H[G[row][col]] - 1
            if med < G[row][col] :
                n = n + 1
            G[row][col] = med
            H[med] = H[med] + 1
    # 对 红色通道 进行中值滤波
    H = np.zeros(256, dtype=int)    # 直方图
    for row in range(1, len(R) - 1) :
        # 到达一个新的行 初始化
        H = np.zeros(256, dtype=int)    # 直方图
        # 求中值
        med = np.uint8(np.median(R[row - 1 : row + 2, 0:3]))
        if med == -128 :
            print(R[row - 1 : row + 2, 0:3])
        n = 0
        for i in range(-1, 2) :
            for j in range(0, 3) :
                H[R[row+i][j]] = H[R[row+i][j]] + 1
                if R[row+i][j] <= med :
                    n = n + 1
        for col in range(1, len(R[row]) - 1) :
            if col == 1 :
                None
            # 移到下一列
            else :
                # 更新直方图 并计算 n 的值
                for i in range(-1, 2) :
                    # 对左列元素 值减一 
                    H[R[row+i][col-2]] = H[R[row+i][col-2]] - 1
                    if R[row+i][col-2] <= med :
                        n = n - 1
                    # 对右列元素 值加一
                    H[R[row+i][col+1]] = H[R[row+i][col+1]] + 1
                    if R[row+i][col+1] <= med :
                        n = n + 1
                # 重新计算中值
                if n > th :
                    while n > th :
                        if med == 0 :
                            break
                        n = n - H[med]
                        med = med - 1
                elif n < th :
                    while n < th :
                        med = med + 1
                        n = n + H[med]
            sum = 0
            # 更新中值后的直方图
            H[R[row][col]] = H[R[row][col]] - 1
            if med < R[row][col] :
                n = n + 1
            R[row][col] = med
            H[med] = H[med] + 1
    
    return cv2.merge([B,G,R])

In [15]:
#测试1
path = './lenna.jpg'
img = cv2.imread(path)
print(img.shape)

(512, 512, 3)


In [16]:
a = medianBlur_turbo(img)
new_path = './lenna_blurred_turbo.jpg'
cv2.imwrite(new_path, a)
cv2.imshow('a', a)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

### 512x512的彩色图片，需要55秒
### 代码可能存在问题，中值滤波图像与上面的不太一样

In [23]:
def medianBlur_turbo1(img, kernel=(3,3), padding_way='ZERO'):
    '''
    img: 图像名称
    kernel is List of List: [m, n] 所要输入的kernel含义不是特别明确，先理解为中值滤波kernel的尺寸
    padding_way a string: REPLICA(填充元像素与边缘像素相同); ZERO(填充像素为0)
    '''
    # 对输入的padding方式进行检查
    padding_way = padding_way.upper()
    allowed_padding = ['ZERO', 'REPLICA']
    if padding_way not in allowed_padding:
        raise ValueError('The padding argument must be one of "ZERO", "REPLICA". Received: ' + str(padding))
    # 设置与img大小相同的全0矩阵，用于接收中值操作后的值，形成中值滤波操作后的结果
    img_after_Blur = np.zeros_like(img)
    # 保存图像的形状
    img_shape = img.shape
    # 图像填充，padding，使用np中的np.lib.pad函数
    padding = ((math.ceil(kernel[0]/2),),(math.ceil(kernel[1]/2),))
    # 若img是三通道的彩色图像，则padding矩阵增加一个维度，但是不做paddding操作
    if len(img_shape) == 3:
        padding = padding + ((0,),)
    #不同的padding方式
    if padding_way == 'REPLICA':
        img = np.lib.pad(img, padding, mode='edge')
    if padding_way == 'ZERO':
        img = np.lib.pad(img, padding, mode='constant', constant_values = 0)
        
    # 以上步骤与原中值滤波函数相同，包括读取pandding方式、进行pandding操作
    th = math.ceil(kernel[0]*kernel[1])
    # 创建保存直方图的数组
    H = np.zeros(256, dtype=int)
    for row in range(1, len(img) - 1):
        # 到达一个新的行，对直方图进行初始化
        H = np.zeros(256, dtype=int)
        # 求中值
        med = np.uint8( np.median(img[row - 1 : row + kernel[0]-1, 0:kernel[1]]) )
        n = 0 
        # 绘制直方图
        for i in range(-1, kernel[0]-1) :
            for j in range(0, kernel[1]) :
                H[img[row+i][j]] = H[img[row+i][j]] + 1
                if img[row+i][j] <= med:
                    n = n + 1
        for col in range(1, len(img[row]) - 1) :
            # 第一列
            if col == 1 :
                None
            # 移到下一列
            else :
                # 更新直方图 并计算 n 的值
                for i in range(-1, kernel[0]-1) :
                    # 对左列元素 值减一 
                    H[img[row+i][col-2]] = H[img[row+i][col-2]] - 1
                    if img[row+i][col-2] <= med :
                        n = n - 1
                    # 对右列元素 值加一
                    H[img[row+i][col+1]] = H[img[row+i][col+1]] + 1
                    if img[row+i][col+1] <= med :
                        n = n + 1
                # 重新计算中值
                if n > 5 :
                    while n > 5 :
                        if med == 0 :
                            break
                        n = n - H[med]
                        med = med - 1
                elif n < th :
                    while n < th :
                        med = med + 1
                        n = n + H[med]
            sum = 0
            for k in range(med + 1) :
                sum = sum + H[k]
            # 更新中值后的直方图
            H[img[row][col]] = H[img[row][col]] - 1
            if med < img[row][col] :
                n = n + 1
            img[row][col] = med
            H[med] = H[med] + 1 
    return img

In [24]:
#测试1
path = './lenna.jpg'
img = cv2.imread(path, 0)
print(img.shape)

(512, 512)


In [25]:
a = medianBlur_turbo1(img)
new_path = './lenna_blurred_turbo1.jpg'
cv2.imwrite(new_path, a)
cv2.imshow('a', a)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

### 512x512的灰度图片，需要30秒